In [ ]:
import collections

import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
from tensorflow import keras
import tensorflow_federated as tff
import pandas as pd
import matplotlib.pyplot as plt



from keras.models import Sequential 
from keras.initializers import HeNormal, GlorotNormal
from keras.layers import Dense
from keras.layers import Dense, Dropout
from keras.metrics import MeanSquaredError, CosineSimilarity, MeanAbsoluteError


TEST_SIZE = 0.2
NUM_CLIENTS = 10
BATCH_SIZE = 512
DROPOUT = 0.1
EPOCHS = 10
PREFETCH_BUFFER = 10
NUM_ROUNDS = 10
UNBALANCED = True
path = os.path.dirname(tff.__file__)
print(path)

np.random.seed(42)
tf.get_logger().setLevel('ERROR')

In [8]:
# Import del dataset e divisione in train e test
train_df = pd.read_csv('datasets/train_BMI.csv')
test_df = pd.read_csv('datasets/test_BMI.csv')

train_x = train_df.drop(columns=['label'])
train_y = train_df['label'].astype('float32').values.reshape(-1, 1)

test_x = test_df.drop(columns=['label'])
test_y = test_df['label'].astype('float32').values.reshape(-1, 1)

# Funzione per il preprocessing dei dati del singolo client che divide il dataset in batch
def preprocess(dataset):
  return dataset.repeat(EPOCHS).batch(BATCH_SIZE).prefetch(PREFETCH_BUFFER)

# Funzione per aggiungere una colonna client_num al dataset in modo tale che ogni client abbia una percentuale di 
# righe del dataset diversa.
def client_unbalanced(dataset, num_clients):
    client_num = []
    prob = np.random.pareto(1, num_clients)
    prob /= np.sum(prob)
    print(prob)
    for i in range(len(dataset)):
        client_num.append(np.random.choice(num_clients, p=prob))
    print(len(client_num))

    print([client_num.count(x) for x in range(num_clients)])
    dataset['client_num'] = client_num
    return dataset

# Funzione per la creazione di un dataset ClientData a partire dal dataset di training a cui viene
# aggiunta una colonna client_num che assegna ad ogni riga un client randomico
def create_clients(dataset, unbalanced, num_clients=NUM_CLIENTS):
    if unbalanced: 
        dataset = client_unbalanced(dataset, num_clients)
    else:
        # Viene creata una lista randomica di client
        client_nums = list(range(num_clients))
        generator = np.random.default_rng(42)
        clients = generator.choice(client_nums, len(dataset))
        dataset['client_num'] = clients

    # Viene convertito il dataset in dizionari, uno per ogni client, con label e pixel associati
    client_train_dataset = collections.OrderedDict()
    grouped_dataset = dataset.groupby('client_num')
    for key, item in grouped_dataset:
        current_client = grouped_dataset.get_group(key)
        data = collections.OrderedDict((('y', train_y), ('x', train_x)))
        client_train_dataset[key] = data

    # I dizionari vengono convertiti in ClientDataset
    def serializable_dataset_fn(client_id):
        client_data = client_train_dataset[client_id]
        return tf.data.Dataset.from_tensor_slices(client_data)

    tff_train_data = tff.simulation.datasets.ClientData.from_clients_and_tf_fn(
        client_ids=list(client_train_dataset.keys()),
        serializable_dataset_fn=serializable_dataset_fn
    )

    return tff_train_data

# Creazione della lista contenente i client con i relativi dataset
elem_spec = {}
def init(dataset, active_clients=NUM_CLIENTS, unbalanced=False): 
    client_data_df = create_clients(dataset, unbalanced, active_clients)
    client_ids = sorted(client_data_df.client_ids)[:active_clients]
    return [preprocess(client_data_df.create_tf_dataset_for_client(x)) for x in client_ids]

In [9]:
def create_keras_model():
  model = Sequential()

  model.add(Dense(128, kernel_initializer=HeNormal(seed=42),input_dim = train_x.shape[1], activation='relu', kernel_regularizer = tf.keras.regularizers.l2(30e-6)))
  model.add(Dropout(DROPOUT))
  model.add(Dense(256, kernel_initializer=HeNormal(seed=42),activation='relu', kernel_regularizer = tf.keras.regularizers.l2(30e-6)))
  model.add(Dropout(DROPOUT))
  model.add(Dense(128, kernel_initializer=HeNormal(seed=42),activation='relu', kernel_regularizer = tf.keras.regularizers.l2(30e-6)))
  model.add(Dense(64, kernel_initializer=HeNormal(seed=42),activation='relu', kernel_regularizer = tf.keras.regularizers.l2(30e-6)))
  model.add(Dense(1, kernel_initializer=GlorotNormal(seed=42),activation='linear'))

  return model

In [10]:
def model_fn():
  keras_model = create_keras_model()
  return tff.learning.models.from_keras_model(
      keras_model,
      input_spec=elem_spec,
      loss=tf.keras.losses.MeanSquaredError(),
      metrics=[MeanSquaredError(), MeanAbsoluteError(), CosineSimilarity()])

In [11]:
def aggregator(algo, prox):
    initial_learning_rate = 0.01
    final_learning_rate = 0.001
    learning_rate_decay_factor = (final_learning_rate / initial_learning_rate)**(1/10)
    steps_per_epoch = int(train_x.shape[0]/BATCH_SIZE)

    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate,
        decay_steps=steps_per_epoch,
        decay_rate=learning_rate_decay_factor,
        staircase=True)

    def client_optimizer_fn(lr):
        return tf.keras.optimizers.Adam(learning_rate=lr)

    if algo == 'weighted avg':
        training_process = tff.learning.algorithms.build_weighted_fed_avg_with_optimizer_schedule(model_fn, 
                                                                                                  client_learning_rate_fn=lr_schedule,   
                                                                                                  client_optimizer_fn=client_optimizer_fn)
    if algo == 'unweighted avg':
        training_process = tff.learning.algorithms.build_unweighted_fed_avg(model_fn, 
                                                                            client_optimizer_fn=tff.learning.optimizers.build_adam(learning_rate=0.001),
                                                                            server_optimizer_fn=tff.learning.optimizers.build_adam(learning_rate=0.001))
    if algo == 'weighted prox':
        training_process = tff.learning.algorithms.build_weighted_fed_prox(model_fn, 
                                                                           proximal_strength=prox, 
                                                                           client_optimizer_fn=tff.learning.optimizers.build_adam(learning_rate=0.001),
                                                                           server_optimizer_fn=tff.learning.optimizers.build_adam(learning_rate=0.001))
    if algo == 'unweighted prox':
        training_process = tff.learning.algorithms.build_weighted_fed_prox(model_fn, 
                                                                           proximal_strength=prox,
                                                                           client_optimizer_fn=tff.learning.optimizers.build_adam(learning_rate=0.001),
                                                                           server_optimizer_fn=tff.learning.optimizers.build_adam(learning_rate=0.001))
    return training_process


In [ ]:
federated_train_data = init(train_df)
elem_spec = federated_train_data[0].element_spec
training_process = aggregator('weighted prox', 20.0)
train_state = training_process.initialize()
for round_num in range(NUM_ROUNDS):
  result = training_process.next(train_state, federated_train_data)
  train_state = result.state
  train_metrics = result.metrics
  print('round {:2d}, metrics={}'.format(round_num, train_metrics))

In [ ]:
print(NUM_ROUNDS)

In [6]:
def keras_evaluate(state, training_process):
  keras_model = create_keras_model()
  keras_model.compile(
      loss=MeanSquaredError(),
      metrics=[MeanSquaredError(), MeanAbsoluteError(), CosineSimilarity()])
  model_weights = training_process.get_model_weights(state)
  model_weights.assign_weights_to(keras_model)
  mse = keras_model.evaluate(x=test_x, y=test_y)
  print('\tEval: mse={a:.3f}, mae={b:.3f}, cs={c:.3f}'.format(a=mse[1], b=mse[2], c=mse[3]))
  return mse[1], mse[2], mse[3]


In [ ]:
keras_evaluate(train_state, training_process)

Esperimenti
==============

***Algoritmo di aggregazione***\

In [ ]:
federated_train_data = init(train_df)
elem_spec = federated_train_data[0].element_spec
# Tuning del parametro di proximal strength
def tune_proximal_strength():
    prox_list = []
    for i in [1.0, 10.0, 20.0, 128.0, 256.0, 512.0]:
        training_process = aggregator('weighted prox', i)
        train_state = training_process.initialize()
        curr = []
        for round_num in range(NUM_ROUNDS):
            result = training_process.next(train_state, federated_train_data)
            train_state = result.state
            train_metrics = result.metrics
            print('round {:2d}, metrics={}'.format(round_num, train_metrics))
            acc_tuple = (round_num, 
                         train_metrics['client_work']['train']['mean_squared_error'], 
                         train_metrics['client_work']['train']['mean_absolute_error'], 
                         train_metrics['client_work']['train']['cosine_similarity'])
            curr.append(acc_tuple)
        prox_list.append((i, curr))

    return prox_list

prox_list = tune_proximal_strength()

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12, 8))
i = 0
rounds = []
mse = []
mae = []
cs = []
for algo, acc_list in prox_list:
    rounds.append([x[0] for x in acc_list])
    mse.append([x[1] for x in acc_list])
    mae.append([x[2] for x in acc_list])
    cs.append([x[3] for x in acc_list])
    i+=1

def plot_metrics(ax, rounds, metrics, labels=[1, 10, 20, 128, 256, 512]):
    for i in range(len(metrics)):
        ax.plot(rounds, metrics[i], label=labels[i])
        ax.legend(loc='lower right')

print(mse[0])
plot_metrics(axs[0, 0], rounds[0], mse)
plot_metrics(axs[0, 1], rounds[1], mae)
plot_metrics(axs[1, 0], rounds[2], cs)

axs[0, 0].set_title('mse')
axs[0, 1].set_title('mae')
axs[1, 0].set_title('cs')

plt.tight_layout()
plt.show()

In [ ]:
federated_train_data = init(train_df, unbalanced=True)
elem_spec = federated_train_data[0].element_spec

def agg_experiment():
    prox_list = []
    for i in ['weighted avg', 'unweighted avg', 'weighted prox', 'unweighted prox']:
        training_process = aggregator(i, 1.0)
        train_state = training_process.initialize()
        curr = []
        for round_num in range(NUM_ROUNDS):
            result = training_process.next(train_state, federated_train_data)
            train_state = result.state
            train_metrics = result.metrics
            print('round {:2d}, metrics={}'.format(round_num, train_metrics))
            acc_tuple = (round_num, 
                         train_metrics['client_work']['train']['mean_squared_error'], 
                         train_metrics['client_work']['train']['mean_absolute_error'], 
                         train_metrics['client_work']['train']['cosine_similarity'])
            curr.append(acc_tuple)
        prox_list.append((i, curr))
    return prox_list

agg_algo_list = agg_experiment()

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12, 8))
i = 0
rounds = []
mse = []
mae = []
cs = []
for algo, acc_list in agg_algo_list:
    rounds.append([x[0] for x in acc_list])
    mse.append([x[1] for x in acc_list])
    mae.append([x[2] for x in acc_list])
    cs.append([x[3] for x in acc_list])
    i+=1

def plot_metrics(ax, rounds, metrics, labels=['weighted avg', 'unweighted avg', 'weighted prox', 'unweighted prox']):
    for i in range(len(metrics)):
        ax.plot(rounds, metrics[i], label=labels[i])
        ax.legend(loc='upper right')

print(mse[0])
plot_metrics(axs[0, 0], rounds[0], mse)
plot_metrics(axs[0, 1], rounds[1], mae)
plot_metrics(axs[1, 0], rounds[2], cs)

axs[0, 0].set_title('mse')
axs[0, 1].set_title('mae')
axs[1, 0].set_title('cs')

plt.tight_layout()
plt.show()

***Numero e Percentuale clients***

In [ ]:
import math
def client_perc_experiment():
    client_list = []
    eval_list = []
    for i in [10, 50, 100, 500]:
        perc_list = []
        for j in [0.25, 0.50, 0.75, 1]:
            federated_train_data = init(train_df, active_clients=math.floor(i*j))
            global elem_spec 
            elem_spec = federated_train_data[0].element_spec
            training_process = aggregator('weighted avg', 1.0)
            train_state = training_process.initialize()
  
            curr = []
            for round_num in range(NUM_ROUNDS):
                result = training_process.next(train_state, federated_train_data)
                train_state = result.state
                train_metrics = result.metrics
                print('round {:2d}, metrics={}'.format(round_num, train_metrics))
                acc_tuple = (round_num, 
                         train_metrics['client_work']['train']['mean_squared_error'], 
                         train_metrics['client_work']['train']['mean_absolute_error'], 
                         train_metrics['client_work']['train']['cosine_similarity'])
                curr.append(acc_tuple)
            eval = keras_evaluate(train_state, training_process)
            eval_list.append((i, j, eval))
            perc_list.append((j, curr))
        client_list.append((i, perc_list))
    return client_list, eval_list

client_list, eval_list = client_perc_experiment()

In [ ]:
print(client_list, eval_list)


In [ ]:
def plot_metric(data, metric_index, metric_name):
    fig, axs = plt.subplots(2, 2, figsize=(15, 10))
    
    for i, (clients, portions_data) in enumerate(data):
        ax = axs[i // 2, i % 2]
        for portion, epoch_data in portions_data:
            epochs = [e[0] for e in epoch_data]
            metric_values = [e[metric_index] for e in epoch_data]
            ax.plot(epochs, metric_values, label=f'{portion*100}% data')
        
        ax.set_title(f'{clients} Clients')
        ax.set_xlabel('Metrics')
        ax.set_ylabel(metric_name)
        ax.legend()
        ax.grid(True) 
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

plot_metric(client_list, 1, 'MSE')
plot_metric(client_list, 2, 'MAE')
plot_metric(client_list, 3, 'CS') 